# MIMIC_Sepsis

## 1. Preparation

To run this document the following requirements must be satisfied:

- Implement the database mimic in **PostgreSQL** and start it. The instruction can be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres). (The name of this environment should be **mimiciv**)
- generate useful abstractions of raw MIMIC-IV data. The instruction be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/concepts_postgres) 



To install all the libraries, run:
```
pip install psycopg2 csv pandas
```


After all the preparation is done, run the following cell to set the connection to the database.
 

In [1]:
import psycopg2
from psycopg2 import sql
import csv
import pandas as pd
import os

# implement the username, password and database name
conn = psycopg2.connect(host='', user='', password='', database='mimiciv')

## 2. Extract selected data from the original database 

According to the paper we need to extract the **state space** and **action space** respectively from the mimiciv database. The table **mimic4 itemid.csv** lists all the items required.

* Here the *IV fluid bolus* of the action space is removed as no data could be found.

#### **Uncomment the following line if you first time run the code**

In [2]:
# uncomment the following line if you first time run the code

# # Read the SQL file

# try:
#     with open('sql_file/selection_patients_cohort.sql', 'r') as file0:
#         sql_script_selection_patients_cohort = file0.read()

#     with open('sql_file/action_from_inputevents.sql', 'r') as file1:
#         sql_script_action = file1.read()
        
#     with open('sql_file/chartevents_dataneeded.sql', 'r') as file2:
#         sql_script_state = file2.read()

#     # Execute the SQL script
#     cursor = conn.cursor()
    
#     cursor.execute(sql.SQL(sql_script_selection_patients_cohort))
#     cursor.execute(sql.SQL(sql_script_action))
#     cursor.execute(sql.SQL(sql_script_state))

#     conn.commit()
#     cursor.close()
    
    
# except (Exception, psycopg2.DatabaseError) as error:
#     print("Error executing SQL statement:", error)

In [3]:
with conn.cursor() as cursor:
    command = "SELECT distinct stay_id FROM mimiciv_derived.sepsis_patients_cohort ;"
    cursor.execute(command)   
    result = cursor.fetchall()
    stay_ids= [row[0] for row in result]
    num_stay_ids = len(stay_ids)
    print('Number of stay_ids: ' + str(num_stay_ids))
    cursor.close()

Number of stay_ids: 6669


## 3. Data transfor
Here we first deal with the action data. 
TODO: here the value per minute is computed, do we need this? (paper4.1,P5)

In [4]:
# generate the dictionary of action
with open('csv/itemid_label_action.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    # Skip the header row
    next(reader)
    # Initialize an empty dictionary and list
    action_label = {}
    a_itemid_list = []
    # Iterate over the rows in the CSV file
    for row in reader:
        # Add the key-value pair to the dictionary
        action_label[row[0]] = row[1]
        # Add the itemid to the list
        a_itemid_list.append(row[0])

with conn.cursor() as cursor:

    for itemid in a_itemid_list:
        command = "select stay_id, starttime, endtime,amount from mimiciv_derived.sepsis_action where itemid={} order by starttime;".format(itemid)
        cursor.execute(command)

        result = cursor.fetchall()
        df = pd.DataFrame(result)
        df.columns = ['stay_id', 'starttime', 'endtime', 'amount']
        
        df['duration'] = df['endtime'] - df['starttime']
        df['duration'] = df['duration'].dt.total_seconds()  # Convert duration to seconds
        df['duration'] = df['duration'] / 60
        df['value_per_minute'] = df['amount'] / df['duration']
        
        os.makedirs('./output/action', exist_ok=True)
        df.to_csv('./output/action/{}.csv'.format(action_label[str(itemid)]), index=0)
        print("output:"+action_label[str(itemid)]+".csv")
    cursor.close()
    

output:Norepinephrine.csv
output:Vasopressin.csv
output:Dobutamine.csv
output:Milrinone.csv
output:Phenylephrine.csv


Then, we treat the **state space** with similar method.

In [101]:

threshold = 1000
itemid_list_state=[]
# generate the dictionary of itemid-abbr
with open('csv/itemid_label_state.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    # Skip the header row
    next(reader)
    # Initialize an empty dictionary and list
    label_state = {}
    itemid_list = []
    # Iterate over the rows in the CSV file
    for row in reader:
        # Add the key-value pair to the dictionary
        label_state[row[0]] = row[1]
        # Add the itemid to the list
        itemid_list.append(row[0])

# Execute the SQL command
with conn.cursor() as cursor:
    
    for itemid in itemid_list:
        
        command_count = "select count(distinct(stay_id)) from mimiciv_derived.sepsis_state where itemid={};".format(itemid)
        cursor.execute(command_count)
        num = cursor.fetchone()[0]
        
        command = "select stay_id, charttime, valuenum from mimiciv_derived.sepsis_state where itemid={} order by charttime;".format(itemid)
        cursor.execute(command)   
        result = cursor.fetchall()
        df=pd.DataFrame(result)
        try:
            df.columns = ['stay_id', 'charttime', 'valuenum']
            df.astype({'stay_id': int, 'charttime': 'datetime64[ns]', 'valuenum': float})
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error executing SQL statement:", error) 
        os.makedirs('./output/state', exist_ok=True)
        
        if (num<num_stay_ids/threshold):
            print("drop:{0:40}".format(label_state[str(itemid)]+".csv")+"\tnumber of stay_id:"+str(num))
            
        else:
            df.to_csv('./output/state/{}.csv'.format(label_state[str(itemid)]),index=0)
            itemid_list_state.append(itemid)
            print("output:{0:40}".format(label_state[str(itemid)]+".csv")+"\tnumber of stay_id:"+str(num))
        
        
        
    cursor.close()
print(itemid_list_state)

output:Heartrate.csv                           	number of stay_id:6669
output:ABPs.csv                                	number of stay_id:2129
output:NBPs.csv                                	number of stay_id:6632
output:ABPd.csv                                	number of stay_id:2131
output:NBPd.csv                                	number of stay_id:6632
output:ABPm.csv                                	number of stay_id:2168
output:NBPm.csv                                	number of stay_id:6632
output:RespiratoryRate.csv                     	number of stay_id:6669
output:TemperatureF.csv                        	number of stay_id:6576
output:TemperatureC.csv                        	number of stay_id:832
output:PH_A.csv                                	number of stay_id:3663
output:PH_V.csv                                	number of stay_id:3184
output:ABE.csv                                 	number of stay_id:3645
output:Hematocrit_serum.csv                    	number of stay_id:6596
output:

## 4.hourly sample

In [104]:
# Set the folder path where the CSV files are stored
folder_path = './output/state/'
columns=['chartdatetime']

# define a new dataframe
df_output = pd.DataFrame(columns=columns)

selected_id = 32768012
#selected_id = 32950566

#FIXME: flag only for test
i=0

# Loop through the file paths and read each file into a DataFrame
for itemid in itemid_list_state:
    
    feature=label_state[str(itemid)]
    path = folder_path + feature+'.csv'
    
    # Load the CSV file into a pandas DataFrame
    dtypes={'stay_id':str,'chartdatetime':str,feature:str}
    df = pd.read_csv(path,names=['stay_id', 'chartdatetime', feature ],dtype=dtypes)
    df['stay_id'] = pd.to_numeric(df['stay_id'], errors='coerce')
    
    # Filter the DataFrame to only the rows from the selected stay_id
    df_filtered = df[df['stay_id'] == selected_id]
    
    # Convert the 'datetime' column to a datetime object
    df_filtered.loc[:,'chartdatetime'] = pd.to_datetime(df_filtered['chartdatetime'].copy())
    #df_filtered['chartdatetime'] = df_filtered['chartdatetime'].apply(pd.to_datetime)

    # Set the 'datetime' column as the DataFrame's index
    df_filtered.set_index('chartdatetime', inplace=True)
    
    # # Resample the DataFrame hourly and forward fill missing values
    df_hourly= df_filtered.resample('H').ffill()
    df_hourly=df_hourly.drop(['stay_id'],axis=1)
    
    df_output['chartdatetime'] = pd.to_datetime(df_output['chartdatetime'])
    #df_output = pd.concat([df_output, df_hourly], join="outer",sort=False)
    df_output=pd.merge(df_output,df_hourly,how='outer',on='chartdatetime')

    
    i+=1
    if(i==50): break

os.makedirs('./output/hourly_sample/', exist_ok=True)
df_output.reset_index().to_csv(f'./output/hourly_sample/stay_id{selected_id}.csv',index=0)

print(i)

# # Reset the index and save the resampled DataFrame to a new CSV file
# df_hourly.reset_index().to_csv('./output/your_resampled_file.csv', index=False, header=None)

/var/folders/05/nx04jp2500xc3gb43lxz42l40000gn/T/ipykernel_25403/1764944956.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'chartdatetime'] = pd.to_datetime(df_filtered['chartdatetime'].copy())
/var/folders/05/nx04jp2500xc3gb43lxz42l40000gn/T/ipykernel_25403/1764944956.py:29: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_filtered.loc[:,'chartdatetime'] = pd.to_datetime(df_filtered['chartdatetime'].copy())
/var/folders/05/nx04jp2500xc3gb43lxz42l40000gn/T/ipykernel_25403/1764944956.py:29: SettingWithCopy

45


/var/folders/05/nx04jp2500xc3gb43lxz42l40000gn/T/ipykernel_25403/1764944956.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'chartdatetime'] = pd.to_datetime(df_filtered['chartdatetime'].copy())
/var/folders/05/nx04jp2500xc3gb43lxz42l40000gn/T/ipykernel_25403/1764944956.py:29: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_filtered.loc[:,'chartdatetime'] = pd.to_datetime(df_filtered['chartdatetime'].copy())
